# Display simulation progress

This notebook illustrates the progressbar classes in pyphysim.

In [ ]:
from time import sleep

import numpy as np
from IPython.display import clear_output, display

from pyphysim.progressbar.progressbar import *
from pyphysim.simulations.results import *
from pyphysim.simulations.runner import *

## Text progressbars

The `ProgressbarText` class is a simple text progressbar that **does not use \r**. That is, the progressbar is not "redraw" at every iteration and instead it only print more characters to the output as the progress increases.
It is particularly suitable if you want to save the progress to a file, instead of showing in a terminal (you can pass a file as the `output` argument of the `__init__` method).

In [ ]:
rep_max = 20
message = "Running some long computation"
pbar = ProgressbarText(rep_max, message=message)
# You can change the width of the progressbar, but only before the first time it is used.
# Furthermore, the value is the highest myultiple of 10 which is lower than or equal to 
# the requested width.
pbar.width = 65  # with 65 the actual width will be 60 characters

for i in range(rep_max + 1):
    sleep(0.1)
    pbar.progress(i)

The `ProgressbarText2` class redraws the whole progressbar as progress changes. You can, for instance, change the message that is shown as progress changes.
It also allow changing the width of the progressbar (even after it has started, but you should not decrease it).

In [ ]:
pbar = ProgressbarText2(rep_max, message=message)
pbar.width = 60  # Notice that this width does not include the message in the side

for i in range(1, rep_max + 1):
    if i == 10:
        pbar.width = 80
    pbar.message = f"Elapsed time: {pbar.elapsed_time}"
    sleep(0.1)
    pbar.progress(i)
    
# This does not update the message because once 100% progress is reached the progressbar stops updating
pbar.message = "Finished"

If you want to display a long message you might prefer the `ProgressbarText3` class. The message is shown in the middle of the progressbar and thus you have more space to increase the size of the progressbar.

In [ ]:
pbar = ProgressbarText3(rep_max)
pbar.width = 80

for i in range(rep_max + 1):
    pbar.message = f"Running some long computation - Elapsed time: {pbar.elapsed_time} - "
    sleep(0.1)
    pbar.progress(i)

When running inside a notebook such as jupyterlab, where widgets from the `ipywidgets` library can be used, you might preffer to use the `ProgressBarIPython` class.

In [ ]:
pbar = ProgressBarIPython(rep_max, message)

for i in range(rep_max + 1):
    pbar.message = f"Elapsed time: {pbar.elapsed_time}"
    sleep(0.1)
    pbar.progress(i)

When working in a notebook the message is displayed using a `Label`, from `ipywidgets` library. That means it supports nice thins such as $\LaTeX$ equations.

In [ ]:
pbar = ProgressBarIPython(rep_max, message="$y = a x^2 + b$")

for i in range(rep_max + 1):
    sleep(0.1)
    pbar.progress(i)

## Display progress in simulators

If you implement a simulator inheritting from `SimulationRunner` then you already get a progressbar and you don't need to create one manually. 
You can change which progressbar class is used to display the progress by changing the value of the `update_progress_function_style` property in your `__init__` method to one of "text1", "text2" (default), or "ipython". Likewise, you can set the message my changing the value of the `progressbar_message` property. If the message some parameter it will be replaced with its value.

In [ ]:
class MyRunner(SimulationRunner):

    def __init__(self):
        SimulationRunner.__init__(self, read_command_line_args=False)
        self.params.add("param 1", [1, 2, 3])
        self.params.add("param 2", "something")
        self.params.set_unpack_parameter("param 1")
        
        self.rep_max = 30
        self.update_progress_function_style = 'ipython'  # 'text1', 'text2' or ipython'
        self.progressbar_message = "param 1 value: {param 1} - param 2: {param 2}"

    def _run_simulation(self, current_parameters):
        sleep(0.1)
        return SimulationResults()


runner = MyRunner()
runner.simulate()

## Distributed progressbar

It is also possible to use a single progressbar to track progress of multiple processes.

For this you create a "server" progressbar, from which you can register "client" progressbars. See the `ProgressbarZMQServer` and `ProgressbarZMQClient` classes.

In [ ]:
from pyphysim.progressbar.progressbar import ProgressbarZMQServer, ProgressbarZMQClient

In [ ]:
port = 10000
server = ProgressbarZMQServer(style='ipython', port=port)
server

In [ ]:
# Register a client
client1 = server.register_client_and_get_proxy_progressbar(10)
client1

In [ ]:
# Register another client
client2 = server.register_client_and_get_proxy_progressbar(10)
client2

Now we can start the progressbar tracking in the server progressbar

In [ ]:
server.start_updater()

**In another process** we can create a client using the information we got when we registered clients.

In [ ]:
c1 = ProgressbarZMQClient(client_id=0, ip="localhost", port=port, finalcount=10)
c1.progress(5)

And yet in another process we create the other client. **Notice that each client has a unique id**.

In [ ]:
c2 = ProgressbarZMQClient(client_id=1, ip="localhost", port=port, finalcount=10)
c2.progress(8)

In [ ]:
# Let's finish the progress in both clients
c1.progress(10)
c2.progress(10)

When implementing simulators inheriting from `SimulationRunner` you automatically get a dsitributed progressbar as above when you call the `simulate_in_parallel` method.

In [ ]:
runner = MyRunner()
runner.progressbar_message = "Running simulation in parallel"

# Note that you need to start the ipyparallel cluster in a terminal with the `ipcluster start` command before the line below is run
runner.simulate_in_parallel()

## Distributed progressbar using multiprocessing

If you are using something such as python `multiprocessing` module for the parallel computation, you might preffer not use ZMQ for the communication between the client progressbars and the server progressbar. In that case, use the `ProgressbarMultiProcessServer` and `ProgressbarMultiProcessClient` classes instead. The following code illustrates the usage.

In [ ]:
import multiprocessing
import numpy as np
from pyphysim.progressbar import ProgressbarMultiProcessServer

# Some "heavy" function to run in diferent proccesses
def func(rep_max, progressbar):
    for i in range(rep_max):
        a = np.random.randn(3,3)
        b = np.random.randn(3,3)
        c = np.linalg.inv(a @ b)
        progressbar.progress(i)
    return c

In [ ]:
# Create the server progressbar
pb = ProgressbarMultiProcessServer(
    message="Running heavy function", style='ipython')

# We will run separated instances of `func` in 4 different processes
num_process = 4

rep_max = 100000
procs = []
for i in range(num_process):
    # The arguments that will be passed to the instance of `func` runing in a 
    # diferent process. Note that we pass the return of 
    # `pb.register_client_and_get_proxy_progressbar`, which is a client progressbar
    proc_args = [
        rep_max,
        pb.register_client_and_get_proxy_progressbar(rep_max)
    ]
    procs.append(
        multiprocessing.Process(target=func,
                                args=proc_args))

# xxxxx Start all processes xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
for proc in procs:
    proc.start()
# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

# xxxxx Start the processbar xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
pb.start_updater()
# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

# xxxxx Join all processes xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
for proc in procs:
    proc.join()
# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

# xxxxx Stop the processbar xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
pb.stop_updater()
# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
